In [1]:
#Importando librerias
import pandas as pd
import numpy as np
import os
import csv


In [2]:
os.getcwd()

'd:\\personal\\DMC\\Python For ETL'

In [3]:
os.listdir("./")

['Archivo_Examen',
 'business-zn',
 'DATA',
 'data.json',
 'Resultados_Clase3',
 'Sesion 2 - 0  Estructuras de datos.ipynb',
 'Sesion 2 - 1 Manejo de String y Ficheros.ipynb',
 'Sesion 2 - 2 Estructuras de control.ipynb',
 'Sesion 3 - 1 Estructuras de control.ipynb',
 'Sesion 3 - 2 Numpy.ipynb',
 'Sesion 3 - 3 Pandas v2.ipynb',
 'Sesion 4 - Examen.ipynb',
 'trusted-zn']

In [4]:
# Ruta de la carpeta
carpeta = 'DATA'

# Fecha de los archivos
fechadata = '20230202'

# Obtener la lista de archivos en la carpeta
archivos = os.listdir(carpeta)

# Filtro los archivos por extensión .csv y nombre que termine en 20230202
archivos_csv = [archivo for archivo in archivos if archivo.endswith(f'{fechadata}.csv')]

# Diccionario para almacenar los DataFrames
dataframes = {}

#print(archivos_csv)
print(f"Se tiene un total de {len(archivos_csv)} archivos csv para cargar ")   


Se tiene un total de 18 archivos csv para cargar 


In [5]:
contadorCarga = 0
# Leemos los archivos CSV y crear los DataFrames
for archivo_csv in archivos_csv:
    ruta_archivo = os.path.join(carpeta, archivo_csv)
    nombre_archivo = os.path.split(archivo_csv)[-1].split(f'_{fechadata}')[0]
    nombre_dataframe = f'DF_{nombre_archivo}'
    try:
        print(ruta_archivo)
        with open(ruta_archivo, 'r', encoding='UTF-8') as file:
            dialect = csv.Sniffer().sniff(file.read(1024))  # Reconocemos el delimitador correcto para cada CSV de manera automática
            file.seek(0)  # Movemos el puntero de archivo al inicio
            reader = csv.reader(file, dialect)  # Creamos un lector de CSV utilizando el dialecto detectado
            header = next(reader)  # Leemos la primera fila como encabezado

            if 'FECCARGA' not in header:  # Verificamos si 'FECCARGA' está presente en el encabezado
                raise ValueError("La columna 'FECCARGA' no está presente")

            df = pd.DataFrame(reader, columns=header)  # Creamos un DataFrame a partir del contenido del CSV
            df.columns = df.columns.str.replace('"', '') # Elimno las comillas dobles de los valores de las columnas
            df = df.replace(to_replace='"', value='', regex=True)

            dataframes[nombre_dataframe] = df  # Agregamos el DataFrame al diccionario de DataFrames

    except (FileNotFoundError, ValueError):
        print(f"Error al leer el archivo: {ruta_archivo}")
    else:
        print(f"Archivo hallado")
        contadorCarga += 1
    finally:
        print(f"Fin de control: {ruta_archivo} cargado correctamente a un DF")


print(f"Se cargaron un total de {contadorCarga} DF")

DATA\CATEGORIA_20230202.csv
Archivo hallado
Fin de control: DATA\CATEGORIA_20230202.csv cargado correctamente a un DF
DATA\CLIENTE_20230202.csv
Archivo hallado
Fin de control: DATA\CLIENTE_20230202.csv cargado correctamente a un DF
DATA\CLIENTE_FECNAC_20230202.csv
Archivo hallado
Fin de control: DATA\CLIENTE_FECNAC_20230202.csv cargado correctamente a un DF
DATA\GENERO_20230202.csv
Archivo hallado
Fin de control: DATA\GENERO_20230202.csv cargado correctamente a un DF
DATA\MEDIOPAGO_20230202.csv
Archivo hallado
Fin de control: DATA\MEDIOPAGO_20230202.csv cargado correctamente a un DF
DATA\MERCADO_20230202.csv
Archivo hallado
Fin de control: DATA\MERCADO_20230202.csv cargado correctamente a un DF
DATA\MODALIDADENVIO_20230202.csv
Archivo hallado
Fin de control: DATA\MODALIDADENVIO_20230202.csv cargado correctamente a un DF
DATA\MODALIDADVENTA_20230202.csv
Archivo hallado
Fin de control: DATA\MODALIDADVENTA_20230202.csv cargado correctamente a un DF
DATA\MONEDA_20230202.csv
Archivo hallado

In [6]:
#Evaluo si todos los DF tiene la columna FECCARGA para luego proceder con la particion
for nombre_dataframe, df in dataframes.items():
    if 'FECCARGA' in df.columns:
        print(f"El DataFrame {nombre_dataframe} tiene la columna 'FECCARGA'")
    else:
        print(f"El DataFrame {nombre_dataframe} no tiene la columna 'FECCARGA'")
        print(f"Columnas del DataFrame {nombre_dataframe}:")
        print(df.columns)

El DataFrame DF_CATEGORIA tiene la columna 'FECCARGA'
El DataFrame DF_CLIENTE tiene la columna 'FECCARGA'
El DataFrame DF_CLIENTE_FECNAC tiene la columna 'FECCARGA'
El DataFrame DF_GENERO tiene la columna 'FECCARGA'
El DataFrame DF_MEDIOPAGO tiene la columna 'FECCARGA'
El DataFrame DF_MERCADO tiene la columna 'FECCARGA'
El DataFrame DF_MODALIDADENVIO tiene la columna 'FECCARGA'
El DataFrame DF_MODALIDADVENTA tiene la columna 'FECCARGA'
El DataFrame DF_MONEDA tiene la columna 'FECCARGA'
El DataFrame DF_PEDIDO tiene la columna 'FECCARGA'
El DataFrame DF_PRIORIDADPEDIDO tiene la columna 'FECCARGA'
El DataFrame DF_PRODUCTO tiene la columna 'FECCARGA'
El DataFrame DF_PROYECCIONVTA tiene la columna 'FECCARGA'
El DataFrame DF_SECTORECONOMICO tiene la columna 'FECCARGA'
El DataFrame DF_SEGMENTO tiene la columna 'FECCARGA'
El DataFrame DF_SUBCATEGORIA tiene la columna 'FECCARGA'
El DataFrame DF_UBIGEO tiene la columna 'FECCARGA'
El DataFrame DF_VENDEDOR tiene la columna 'FECCARGA'


In [7]:
# Recorremos cada DataFrame en la lista de diccionarios
for nombre_dataframe, df in dataframes.items():
    if 'FECCARGA' in df.columns:
        # Imprimimos algunos registros antes de la transformación
        print(f"Registros antes de la transformación en {nombre_dataframe}:")
        print(df['FECCARGA'].head())
        
        # Convertimos la columna 'FECCARGA' a formato de fecha
        df['FECCARGA'] = pd.to_datetime(df['FECCARGA'])
        
        # Agregamos columnas adicionales de año, mes y día
        df['ANIO'] = df['FECCARGA'].dt.year
        df['MES'] = df['FECCARGA'].dt.month
        df['DIA'] = df['FECCARGA'].dt.day
        
        # Obtenemos solo el año, mes, día, horas y minutos
        df['FECCARGA'] = df['FECCARGA'].dt.strftime('%Y-%m-%d %H:%M')
        
        # Imprimimos algunos registros después de la transformación
        print(f"Registros después de la transformación en {nombre_dataframe}:")
        print(df[['FECCARGA', 'ANIO', 'MES', 'DIA']].head())


Registros antes de la transformación en DF_CATEGORIA:
0    2023-02-02
1    2023-02-02
2    2023-02-02
Name: FECCARGA, dtype: object
Registros después de la transformación en DF_CATEGORIA:
           FECCARGA  ANIO  MES  DIA
0  2023-02-02 00:00  2023    2    2
1  2023-02-02 00:00  2023    2    2
2  2023-02-02 00:00  2023    2    2
Registros antes de la transformación en DF_CLIENTE:
0    2023-02-02 00:00:00.0000000
1    2023-02-02 00:00:00.0000000
2    2023-02-02 00:00:00.0000000
3    2023-02-02 00:00:00.0000000
4    2023-02-02 00:00:00.0000000
Name: FECCARGA, dtype: object
Registros después de la transformación en DF_CLIENTE:
           FECCARGA  ANIO  MES  DIA
0  2023-02-02 00:00  2023    2    2
1  2023-02-02 00:00  2023    2    2
2  2023-02-02 00:00  2023    2    2
3  2023-02-02 00:00  2023    2    2
4  2023-02-02 00:00  2023    2    2
Registros antes de la transformación en DF_CLIENTE_FECNAC:
0    2023-02-02 00:00:00
1    2023-02-02 00:00:00
2    2023-02-02 00:00:00
3    2023-02-02 0

In [8]:
# Nombre de los directorios que vamos a crear
directorios = ["trusted-zn", "business-zn"]

# Ruta base donde se crearán los directorios
ruta_base = os.getcwd() 

# Crear los directorios
for directorio in directorios:
    ruta_directorio = os.path.join(ruta_base, directorio)
    os.makedirs(ruta_directorio, exist_ok=True) #creamos la ruta y de ya exisitir no generamos ninguna excepcion solo continua el script
    print(f"Directorio '{directorio}' creado en '{ruta_directorio}'")

Directorio 'trusted-zn' creado en 'd:\personal\DMC\Python For ETL\trusted-zn'
Directorio 'business-zn' creado en 'd:\personal\DMC\Python For ETL\business-zn'


In [9]:

carpeta_trusted_zn = os.path.join(os.getcwd(), 'trusted-zn')
try:
    os.chdir(carpeta_trusted_zn)
except:
    pass




In [10]:
inTrust = False
if os.getcwd() == 'd:\\personal\\DMC\Python For ETL\\trusted-zn':
    inTrust = True
    print('Dentro de la carpeta trst')


Dentro de la carpeta trst


In [11]:
for nombre_dataframe, df in dataframes.items():
    if inTrust == True:
        # Obtener el nombre de la carpeta
        nombre_carpeta = 'trs' + nombre_dataframe[2:]
        print(nombre_carpeta)
        try:
            os.mkdir(nombre_carpeta)
            #os.chdir(os.getcwd)
        except:
            print(f"No se pudo crear la carpeta {os.getcwd} \\ {nombre_carpeta}")

trs_CATEGORIA
No se pudo crear la carpeta <built-in function getcwd> \ trs_CATEGORIA
trs_CLIENTE
No se pudo crear la carpeta <built-in function getcwd> \ trs_CLIENTE
trs_CLIENTE_FECNAC
No se pudo crear la carpeta <built-in function getcwd> \ trs_CLIENTE_FECNAC
trs_GENERO
No se pudo crear la carpeta <built-in function getcwd> \ trs_GENERO
trs_MEDIOPAGO
No se pudo crear la carpeta <built-in function getcwd> \ trs_MEDIOPAGO
trs_MERCADO
No se pudo crear la carpeta <built-in function getcwd> \ trs_MERCADO
trs_MODALIDADENVIO
No se pudo crear la carpeta <built-in function getcwd> \ trs_MODALIDADENVIO
trs_MODALIDADVENTA
No se pudo crear la carpeta <built-in function getcwd> \ trs_MODALIDADVENTA
trs_MONEDA
No se pudo crear la carpeta <built-in function getcwd> \ trs_MONEDA
trs_PEDIDO
No se pudo crear la carpeta <built-in function getcwd> \ trs_PEDIDO
trs_PRIORIDADPEDIDO
No se pudo crear la carpeta <built-in function getcwd> \ trs_PRIORIDADPEDIDO
trs_PRODUCTO
No se pudo crear la carpeta <built-i

In [12]:
# Obtener la lista de directorios creados
lista_directorios = [nombre for nombre in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(os.getcwd(), nombre))]

# Imprimir la lista de directorios
print("Lista de directorios:")
for directorio in lista_directorios:
    #print(directorio)
    print(directorio[4:])

Lista de directorios:
CATEGORIA
CLIENTE
CLIENTE_FECNAC
GENERO
MEDIOPAGO
MERCADO
MODALIDADENVIO
MODALIDADVENTA
MONEDA
PEDIDO
PRIORIDADPEDIDO
PRODUCTO
PROYECCIONVTA
SECTORECONOMICO
SEGMENTO
SUBCATEGORIA
UBIGEO
VENDEDOR


In [13]:
NombreCarpeta = ''
partition_cols = ['ANIO', 'MES', 'DIA']

#Guardamos en formato parquet segun cada directorio
for nombre_directorio in os.listdir(os.getcwd()):
    ruta_directorio = os.path.join(os.getcwd(), nombre_directorio)
    NombreCarpeta = ruta_directorio.split("\\")[-1]
    NombreCarpeta = NombreCarpeta[4:]
    #print(ruta_directorio)
    #print(NombreCarpeta)
    for nombre_dataframe, dataframe in dataframes.items():
        if (NombreCarpeta == nombre_dataframe[3:]):
            #print(dataframe)
            print(ruta_directorio)
            dfinal = dataframe.to_parquet(ruta_directorio,partition_cols=partition_cols, compression='gzip')
            print('exportado')

   

d:\personal\DMC\Python For ETL\trusted-zn\trs_CATEGORIA
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_CLIENTE
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_CLIENTE_FECNAC
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_GENERO
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_MEDIOPAGO
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_MERCADO
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_MODALIDADENVIO
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_MODALIDADVENTA
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_MONEDA
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_PEDIDO
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_PRIORIDADPEDIDO
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_PRODUCTO
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_PROYECCIONVTA
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_SECTORECONOMICO
exportado
d:\personal\DMC\Python For ETL\trusted-zn\trs_SEGMENTO
expo

In [14]:
# leer en formato parquet
dfCategoria = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_CATEGORIA')  
dfCliente = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_CLIENTE')  
dfCliente_FecNac = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_CLIENTE_FECNAC')  
dfGenero = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_GENERO')  
dfMedioPago = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_MEDIOPAGO')  
dfMercado = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_MERCADO')  
dfModalidadEnvio = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_MODALIDADENVIO')  
dfModalidadVenta = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_MODALIDADVENTA')  
dfMoneda = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_MONEDA')  
dfPedido = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_PEDIDO')  
dfPrioridadPedido = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_PRIORIDADPEDIDO')  
dfProducto = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_PRODUCTO')  
dfProyeccionVenta = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_PROYECCIONVTA')
dfSectorEconomico = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_SECTORECONOMICO')
dfSegmento = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_SEGMENTO')  
dfSubCategoria = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_SUBCATEGORIA')  
dfUbigeo = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_UBIGEO')  
dfVendedor = pd.read_parquet(r'd:\personal\DMC\Python For ETL\trusted-zn\trs_VENDEDOR')  

In [15]:
df_Dimension_Producto = pd.merge(left=dfProducto,right=dfSubCategoria, how = 'left', left_on='CODSUBCAT', right_on='CODSUBCAT',suffixes=('', '_right')) 
df_Dimension_Producto = df_Dimension_Producto.filter(regex='^(?!.*_right$)') #elimine aquellas columnas duplicadas marcadas con el sufijo
df_Dimension_Producto = pd.merge(left=df_Dimension_Producto,right=dfCategoria, how = 'left', left_on='CODCAT', right_on='CODCAT',suffixes=('', '_right')) 
df_Dimension_Producto = df_Dimension_Producto.filter(regex='^(?!.*_right$)') #elimine aquellas columnas duplicadas marcadas con el sufijo
df_Dimension_Producto = df_Dimension_Producto.drop(['ANIO','MES','DIA'], axis=1)
df_Dimension_Producto.head(3)


,IDPRODUCTO,CODPROD,NOMPROD,CODMNDA,CODSUBCAT,MTOPRECUNIT,FECCARGA,CODCAT,NOMSUBCAT,NOMCAT
0,1,FUR-BO-3174,"Atlantic Metals Mobile 2-Shelf Bookcases, Cust...",2,8,72.294,2023-02-02 00:00,2,Librerías,Mobiliario
1,2,FUR-BO-3175,"Atlantic Metals Mobile 3-Shelf Bookcases, Cust...",2,8,782.940,2023-02-02 00:00,2,Librerías,Mobiliario
2,3,FUR-BO-3176,"Atlantic Metals Mobile 4-Shelf Bookcases, Cust...",2,8,477.666,2023-02-02 00:00,2,Librerías,Mobiliario


In [16]:
df_Dimension_Moneda = dfMoneda.drop(['ANIO','MES','DIA'], axis=1)
df_Dimension_Moneda.head(3)

,CODMNDA,DESMNDA,DESISO,DESSIMB,FECCARGA
0,1,Soles,PEN,S/,2023-02-02 00:00
1,2,Dolares Americanos,USD,$,2023-02-02 00:00
2,3,Euros,EUR,€,2023-02-02 00:00


In [17]:
df_Dimension_Cliente = pd.merge(left=dfCliente,right=dfMercado, how = 'left', left_on='CODMRCADO', right_on='CODMRCADO',suffixes=('', '_right')) 
df_Dimension_Cliente = df_Dimension_Cliente.filter(regex='^(?!.*_right$)') #elimine aquellas columnas duplicadas marcadas con el sufijo
df_Dimension_Cliente = pd.merge(left=df_Dimension_Cliente,right=dfSectorEconomico, how = 'left', left_on='CODSECTECON', right_on='CODSECTECON',suffixes=('', '_right')) 
df_Dimension_Cliente = df_Dimension_Cliente.filter(regex='^(?!.*_right$)') #elimine aquellas columnas duplicadas marcadas con el sufijo
df_Dimension_Cliente = pd.merge(left=df_Dimension_Cliente,right=dfUbigeo, how = 'left', left_on='CODUBIGEO', right_on='CODUBIGEO',suffixes=('', '_right')) 
df_Dimension_Cliente = df_Dimension_Cliente.filter(regex='^(?!.*_right$)') #elimine aquellas columnas duplicadas marcadas con el sufijo
df_Dimension_Cliente = pd.merge(left=df_Dimension_Cliente,right=dfCliente_FecNac, how = 'left', left_on='CODCLI', right_on='COCLI',suffixes=('', '_right')) 
df_Dimension_Cliente = df_Dimension_Cliente.filter(regex='^(?!.*_right$)')
df_Dimension_Cliente = df_Dimension_Cliente.drop(['COCLI','IDCLI'], axis=1)
df_Dimension_Cliente = pd.merge(left=df_Dimension_Cliente,right=dfGenero, how = 'left', left_on='CODGENERO', right_on='IDGENERO',suffixes=('', '_right')) 
df_Dimension_Cliente = df_Dimension_Cliente.filter(regex='^(?!.*_right$)')
df_Dimension_Cliente = df_Dimension_Cliente.drop(['ANIO','MES','DIA','NOMBRES'], axis=1)
df_Dimension_Cliente.head(3)

,NOMCLI,CODCLI,CODMRCADO,CODSECTECON,CODUBIGEO,CODGENERO,FECCARGA,DESMERCADO,CODSGMNTO,DESSECTECON,CODPOSTAL,NOMREGION,NOMPAIS,NOMESTADO,NOMCIUDAD,﻿IDCLI,FECHANAC,IDGENERO,DESGENERO
0,"CEVALLOS ASENJO, MIGUEL LEONARDO",AA-10315102,2,0,1087,1,2023-02-02 00:00,Asia-Pacífico,1,CONSUMIDOR FINAL,.,Asia Sudoriental,Filipinas,Capital Nacional,Marikina,1,1993-04-29 00:00:00,1,Masculino
1,"MOLERO CASTELLARES, ABEL MARIANO",AA-10315120,3,0,1789,1,2023-02-02 00:00,Europa,1,CONSUMIDOR FINAL,.,Europa Meridional,España,Madrid,Parla,2,1976-12-12 00:00:00,1,Masculino
2,"GASLAC CHASQUIBOL, ADALICIO",AA-10315139,3,0,2884,1,2023-02-02 00:00,Europa,1,CONSUMIDOR FINAL,.,Europa Septentrional,Reino Unido,Inglaterra,Garforth,3,1976-05-16 00:00:00,1,Masculino


In [18]:
df_Dimension_ModalidadVenta = dfModalidadVenta.drop(['ANIO','MES','DIA'], axis=1)
df_Dimension_ModalidadVenta.head(3)

,CODMODVALVTA,DESMODVALVTA,FECCARGA
0,1,Venta Online,2023-02-02 00:00
1,2,Venta Presencial,2023-02-02 00:00


In [19]:
df_Dimension_MedioPago = dfMedioPago.drop(['ANIO','MES','DIA'], axis=1)
dfMedioPago.head(3)

,CODMEDIOPAGO,DESMEDIOPAGO,FECCARGA,ANIO,MES,DIA
0,1,Efectivo Contra Entrega,2023-02-02 00:00,2023,2,2
1,2,Tarjeta de debito,2023-02-02 00:00,2023,2,2
2,3,Tarjera de Crédito,2023-02-02 00:00,2023,2,2


In [20]:
df_Dimension_Vendedor = dfVendedor.drop(['ANIO','MES','DIA'], axis=1)
df_Dimension_Vendedor.head(3)

,CODVEND,NOMVEND,CODMNDA,MTOSUELDOBASE,PCTCOMIS,FECCARGA
0,0,Autoventa,1,0.00,0.00,2023-02-02 00:00
1,1,Frank Llañes,1,1500.00,0.05,2023-02-02 00:00
2,2,Christian Peña,1,1500.00,0.08,2023-02-02 00:00


In [21]:
df_Dimension_PrioridadPedido = dfPrioridadPedido.drop(['ANIO','MES','DIA'], axis=1)
df_Dimension_PrioridadPedido.head(3)

,CODPRIORPEDI,DESPRIORPEDI,FECCARGA
0,1,Alta,2023-02-02 00:00
1,2,Baja,2023-02-02 00:00
2,3,Crítica,2023-02-02 00:00


In [24]:
try:
    os.makedirs(r'd:\personal\DMC\Python For ETL\business-zn\Dim_Producto',exist_ok = True)
    os.makedirs(r'd:\personal\DMC\Python For ETL\business-zn\Dim_Moneda',exist_ok = True)
    os.makedirs(r'd:\personal\DMC\Python For ETL\business-zn\Dim_Cliente',exist_ok = True)
    os.makedirs(r'd:\personal\DMC\Python For ETL\business-zn\Dim_ModalidadVenta',exist_ok = True)
    os.makedirs(r'd:\personal\DMC\Python For ETL\business-zn\Dim_MedioPago',exist_ok = True)
    os.makedirs(r'd:\personal\DMC\Python For ETL\business-zn\Dim_Vendedor',exist_ok = True)
    os.makedirs(r'd:\personal\DMC\Python For ETL\business-zn\Dim_PrioridadPedido',exist_ok = True)
except:
    print("Error no se pudieron crear las carpetas en la capa business")
else:
    print("Se termino la creacion de carpetas para las dimensiones")
finally:
    print("Fin")


Se termino la creacion de carpetas para las dimensiones
Fin


In [31]:
#Profesor aqui no sé porque me comenzo a dar error de permisos si arriba habia podido crear sin problemas, hasta donde veo la sentencia esta correcta
os.getcwd()
df_Dimension_PrioridadPedido.to_parquet(r'd:\personal\DMC\Python For ETL\business-zn\Dim_PrioridadPedido', compression='gzip')
df_Dimension_Moneda.to_parquet(r'd:\personal\DMC\Python For ETL\business-zn\Dim_Moneda', compression='gzip')
df_Dimension_Cliente.to_parquet(r'd:\personal\DMC\Python For ETL\business-zn\Dim_Cliente', compression='gzip')
df_Dimension_ModalidadVenta.to_parquet(r'd:\personal\DMC\Python For ETL\business-zn\Dim_ModalidadVenta', compression='gzip')
df_Dimension_MedioPago.to_parquet(r'd:\personal\DMC\Python For ETL\business-zn\Dim_MedioPago', compression='gzip')
df_Dimension_Vendedor.to_parquet(r'd:\personal\DMC\Python For ETL\business-zn\Dim_Vendedor', compression='gzip')
df_Dimension_PrioridadPedido.to_parquet(r'd:\personal\DMC\Python For ETL\business-zn\Dim_PrioridadPedido', compression='gzip')



PermissionError: [Errno 13] Permission denied: 'd:\\personal\\DMC\\Python For ETL\\business-zn\\Dim_PrioridadPedido'

In [34]:
dfPedido.head(3)

,CODPEDID,FECPEDID,FECENV,CODCLI,CODPROD,MTOVALUNIT,CTDPEDID,MTOSUBT,MTODSCTO,MTOBENEF,...,CODVEND,CODMNDA,IDCLI,IDPRODUCTO,CODMODALENV,CODPRIORPEDI,FECCARGA,ANIO,MES,DIA
0,CA-2015-BD11620140-42005,2021-01-01 00:00:00.0000000,2021-01-06 00:00:00.0000000,BD-116201404,TEC-PH-3017,302.38,3,907.13,0.20,22.68,...,3,2,1534,3486,1,4,2023-02-02 00:00,2023,2,2
1,IN-2015-KN164507-42005,2021-01-01 00:00:00.0000000,2021-01-05 00:00:00.0000000,KN-164507,TEC-MA-5008,846.29,3,2538.86,0.10,47.00,...,5,2,9658,3370,1,4,2023-02-02 00:00,2023,2,2
2,IT-2015-AS1024045-42005,2021-01-01 00:00:00.0000000,2021-01-03 00:00:00.0000000,AS-1024045,TEC-PH-5349,332.16,6,1992.98,0.15,-43.14,...,0,2,1205,3675,3,3,2023-02-02 00:00,2023,2,2


In [36]:
df_Fact_Pedido = pd.merge(left=dfPedido,right=df_Dimension_Cliente, how = 'left', left_on='CODCLI', right_on='CODCLI',suffixes=('', '_right')) 
df_Fact_Pedido = df_Fact_Pedido.filter(regex='^(?!.*_right$)')
df_Fact_Pedido = pd.merge(left=dfPedido,right=df_Dimension_Producto, how = 'left', left_on='CODPROD', right_on='CODPROD',suffixes=('', '_right')) 
df_Fact_Pedido = df_Fact_Pedido.filter(regex='^(?!.*_right$)')
df_Fact_Pedido.head(5)

,CODPEDID,FECPEDID,FECENV,CODCLI,CODPROD,MTOVALUNIT,CTDPEDID,MTOSUBT,MTODSCTO,MTOBENEF,...,FECCARGA,ANIO,MES,DIA,NOMPROD,CODSUBCAT,MTOPRECUNIT,CODCAT,NOMSUBCAT,NOMCAT
0,CA-2015-BD11620140-42005,2021-01-01 00:00:00.0000000,2021-01-06 00:00:00.0000000,BD-116201404,TEC-PH-3017,302.38,3,907.13,0.20,22.68,...,2023-02-02 00:00,2023,2,2,Adtran 1202752G1,3,201.584,1,Teléfonos,Tecnología
1,IN-2015-KN164507-42005,2021-01-01 00:00:00.0000000,2021-01-05 00:00:00.0000000,KN-164507,TEC-MA-5008,846.29,3,2538.86,0.10,47.00,...,2023-02-02 00:00,2023,2,2,"Konica Inkjet, Wireless",4,1253.760,1,Máquinas,Tecnología
2,IT-2015-AS1024045-42005,2021-01-01 00:00:00.0000000,2021-01-03 00:00:00.0000000,AS-1024045,TEC-PH-5349,332.16,6,1992.98,0.15,-43.14,...,2023-02-02 00:00,2023,2,2,"Nokia Office Telephone, with Caller ID",3,195.390,1,Teléfonos,Tecnología
3,CA-2015-KL16555140-42005,2021-01-01 00:00:00.0000000,2021-01-07 00:00:00.0000000,KL-165551404,OFF-BI-5093,159.77,7,1118.38,0.20,53.92,...,2023-02-02 00:00,2023,2,2,Lock-Up Easel 'Spel-Binder',10,34.236,3,Archivadores,Material de oficina
4,RO-2015-LW6825107-42005,2021-01-01 00:00:00.0000000,2021-01-04 00:00:00.0000000,LW-6825107,OFF-AR-6106,99.48,2,198.96,0.00,23.82,...,2023-02-02 00:00,2023,2,2,"Stanley Canvas, Easy-Erase",11,298.440,3,Arte,Material de oficina
